<a href="https://colab.research.google.com/github/aminfadaei116/Deep-Learning-Course/blob/master/CA1/Q8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
from sklearn.utils import shuffle

In [ ]:
Test_data = pd.read_csv('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/Fashion-MNIST/Reduced_Test_Data.csv', header = None)
Test_label = pd.read_csv('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/Fashion-MNIST/testLabels.csv', header = None)
Train_data = pd.read_csv('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/Fashion-MNIST/Reduced_Train_Data.csv', header = None)
Train_label = pd.read_csv('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/Fashion-MNIST/trainLabels.csv', header = None)

In [ ]:
Test_data = Test_data.to_numpy()
Test_label = Test_label.to_numpy()
Train_data = Train_data.to_numpy()
Train_label = Train_label.to_numpy()

In [ ]:
maxD = np.max(Train_data, axis=0)
minD = np.min(Train_data, axis=0)
delta = maxD - minD
Train_data = Train_data - minD
Test_data = Test_data - minD
Train_data = 2 * (Train_data/delta) - 1
Test_data = 2 * (Test_data/delta) - 1

In [ ]:
num = 200
noise_data = np.random.normal(loc=0, scale=0.2, size=(num, len(Train_data[0])))
noise_label = 10 * np.ones(shape=(num,1))
Train_data = np.concatenate((Train_data, noise_data))
Train_label = np.concatenate((Train_label, noise_label))
Train_data, Train_label =shuffle(Train_data, Train_label, random_state=0)
Train_label = (Train_label).astype(int)

In [ ]:
def relu(x):
  return np.maximum(x, np.zeros_like(x))

def Cal_accuracy(True_label, Estimated_label):
  return np.count_nonzero(np.transpose(True_label)==np.array(Estimated_label))/True_label.size

def step(x):
  return 1*(x > np.zeros_like(x))

def encode(vect, idx, landa):
  temp = np.multiply(step(landa - vect),-1)
  if idx != 10:
    temp[idx] = 1
  return temp

def gelu(x):
    return 0.5 * np.multiply(x, 1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

def Dgelu(x):
  return np.divide(0.053516 * np.power(x, 3) + np.multiply(0.398942, x) , np.power(np.cosh(0.0356774 * np.power(x, 3) +np.multiply(0.79788, x)),2)) + 0.5 * ( 1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

def RBF_encode(data, number_class, dim):
  data = np.reshape(data, (dim,int(data.size/dim)), order='F')
  data = LA.norm(data,1, axis = 0)
  data = np.reshape(data, (number_class, int(data.size/number_class)), order='F')
  return data

def expand(x):
  temp = []
  for i in range(len(x)):
    temp.append(x[i])
    temp.append(x[i])
  return temp

In [ ]:
class Network:
  def __init__(self, input_node, hidden_layer, hidden_node, output_node):
    self.number_input_node = input_node
    self.number_hidden_layer = hidden_layer
    self.number_hidden_node = hidden_node
    self.number_class_output = output_node
    self.distance_dim = 2
    self.weight = {}
    self.layer = {}
    self.batch_data = {}
    self.batch_label = {}
    self.diff_weight = {}
    self.score = []
    self.result = []
    self.init_weight()
    self.loss = []
    self.back_prob_output = []
    for i in range(self.number_hidden_layer + 1):
      self.diff_weight["w" + str(i) + "_prev"] = 0
      self.diff_weight["b" + str(i) + "_prev"] = 0
    self.momentum = 0.9
    self.learning_rate = 0.001
    self.number_turn = 100
    self.batchS = 128
    self.accuracy_graph = []
    self.loss_graph = []
    self.landa = 50
    self.attack_rate = 0.5

  def set_turn(self, x):
    self.number_turn = x

  def set_learning_rate(self, x):
    self.learning_rate = x

  def set_batch_size(self, x):
    self.batchS = x
  
  def load_weights(self):
    for i in range(self.number_hidden_layer + 1):
      self.weight["w" + str(i)] = (pd.read_csv("/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/Fashion-MNIST/w" + str(i) + ".csv", header=None)).to_numpy()
      self.weight["b" + str(i)] = (pd.read_csv("/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/Fashion-MNIST/b" + str(i) + ".csv", header=None)).to_numpy()

  def init_weight(self):
    self.weight["w0"] = np.random.normal(loc=0, scale=0.14, size=(self.number_hidden_node[0], self.number_input_node))
    self.weight["b0"] = np.random.normal(loc=0, scale=0.14, size=(self.number_hidden_node[0], 1))
    for i in range(self.number_hidden_layer - 1):
      self.weight["w" + str(i+1)] = (np.random.normal(loc=0, scale=0.14, size=(self.number_hidden_node[i+1], self.number_hidden_node[i])))
      self.weight["b" + str(i+1)] = (np.random.normal(loc=0, scale=0.14, size=(self.number_hidden_node[i+1], 1)))
    self.weight["w" + str(self.number_hidden_layer)] = np.random.normal(loc=0, scale=0.14, size=(self.number_class_output * self.distance_dim, self.number_hidden_node[self.number_hidden_layer-1]))
    self.weight["b" + str(self.number_hidden_layer)] = np.random.normal(loc=0, scale=0.14, size=(self.number_class_output * self.distance_dim, 1))
    
  def forward(self, data):
    self.layer.clear()
    self.layer["input"] = data
    if len(data) == data.size:
      self.layer["z0"] = (np.matmul(self.weight["w0"],np.transpose(data))).reshape((len(MyNetwork.weight["w0"]),1))
    else:
      self.layer["z0"] = (np.matmul(self.weight["w0"], np.transpose(data)))
    self.layer["z0"] = np.add(self.layer["z0"], self.weight["b0"])
    self.layer["r0"] = gelu(self.layer["z0"])
    self.layer["s0"] = Dgelu(self.layer["z0"])
    for i in range(self.number_hidden_layer - 1):
      self.layer["z" + str(i + 1)] = np.matmul(self.weight["w" + str(i + 1)], self.layer["r" + str(i)])
      self.layer["z" + str(i + 1)] = np.add(self.layer["z" + str(i + 1)], self.weight["b" + str(i + 1)])
      self.layer["r" + str(i + 1)] = gelu(self.layer["z" + str(i + 1)])
      self.layer["s" + str(i + 1)] = Dgelu(self.layer["z" + str(i + 1)])
    self.layer["z" + str(self.number_hidden_layer)] = np.matmul(self.weight["w" + str(self.number_hidden_layer)], self.layer["r" + str(self.number_hidden_layer - 1)])
    self.layer["z" + str(self.number_hidden_layer)] = np.add(self.layer["z" + str(self.number_hidden_layer)], self.weight["b" + str(self.number_hidden_layer)])
    self.layer["r" + str(self.number_hidden_layer)] = RBF_encode(self.layer["z" + str(self.number_hidden_layer)], self.number_class_output, self.distance_dim)
    self.layer["s" + str(self.number_hidden_layer)] = np.sign(self.layer["z" + str(self.number_hidden_layer)])
    self.score = self.layer["r" + str(self.number_hidden_layer)]
    #print(np.transpose(self.score))
    temp = self.decide_RBF(self.score, self.landa)
    return temp
  
  def decide_RBF(self, data, landa):
    self.result = np.argmin(data, axis=0)
    self.result[np.min(data, axis=0)>landa] = 10
    return self.result

  def cal_loss(self, label):
    self.loss.clear()
    self.back_prob_output = []
    temp = np.transpose(self.score)
    for i in range(len(label)):
      if label[i] != 10:
        self.loss.append(np.sum(relu((self.landa - temp[i]))) - relu(self.landa- temp[i][label[i]]) +   temp[i][int(label[i])])
      else:
        self.loss.append(np.sum(relu(self.landa - temp[i])))
      self.back_prob_output.append(encode(temp[i], label[i], self.landa))
    self.back_prob_output = np.transpose(self.back_prob_output)/len(label)
    total_loss = np.sum(self.loss)/len(self.loss)
    self.loss_graph.append(total_loss)
    return total_loss

  def batching(self, data, label):
    batch_size = self.batchS
    if len(data) == data.size:
      batch[0] = data
      return
    for i in range(int(len(data)/batch_size)):
      self.batch_data[i] = data[i*batch_size:(i+1)*batch_size]
      self.batch_label[i] = label[i*batch_size:(i+1)*batch_size]
    if len(data)%batch_size != 0:
      self.batch_data[int(len(data)/batch_size)] = data[int(len(data)/batch_size)*batch_size:len(data)]
      self.batch_label[int(len(data)/batch_size)] = label[int(len(data)/batch_size)*batch_size:len(data)]

  def back_prob(self):
    temp = np.multiply(expand(self.back_prob_output), self.layer["s" + str(self.number_hidden_layer)])
    for i in range(self.number_hidden_layer):
      self.diff_weight["w" + str(self.number_hidden_layer - i)] = np.matmul(temp, np.transpose(self.layer["r"+ str(self.number_hidden_layer-1-i)]))
      self.diff_weight["b" + str(self.number_hidden_layer - i)] = np.matmul(temp, np.ones((len(temp[0]), 1)))
      temp = np.matmul(np.transpose(self.weight["w" + str(self.number_hidden_layer - i)]), temp)
      temp = np.multiply(self.layer["s" + str(self.number_hidden_layer - i - 1)], temp)
    self.diff_weight["w0"] = np.matmul(temp, self.layer["input"])
    self.diff_weight["b0"] = np.matmul(temp, np.ones((len(temp[0]), 1)))

    for i in range(self.number_hidden_layer + 1):
      self.diff_weight["w" + str(i) + "_prev"] = ((1 - self.momentum) * self.diff_weight["w" + str(i)] + self.momentum * self.diff_weight["w" + str(i) + "_prev"])
      self.diff_weight["b" + str(i) + "_prev"] = ((1 - self.momentum) * self.diff_weight["b" + str(i)] + self.momentum * self.diff_weight["b" + str(i) + "_prev"])

    for i in range(self.number_hidden_layer + 1):
      self.weight["w" + str(i)] = self.weight["w" + str(i)] - self.diff_weight["w" + str(i) + "_prev"] * self.learning_rate
      self.weight["b" + str(i)] = self.weight["b" + str(i)] - self.diff_weight["b" + str(i) + "_prev"] * self.learning_rate

  def Train_Network(self, data, label):
    epoch = 0
    for j in range(self.number_turn):
      self.batching(data, label)
      for i in range(len(self.batch_data)):
        self.forward(self.batch_data[i])
        self.accuracy_graph.append(Cal_accuracy(self.batch_label[i], self.forward(self.batch_data[i])))
        print(self.cal_loss(self.batch_label[i]), " step ", i," of the epoch ", j , "the steps is: ", epoch)
        self.back_prob()
        epoch += 1

  def Test_Network(self, data, label):
    if(len(data) == data.size):
      temp = self.batchS
      self.batchS = 1
      print(Cal_accuracy(label, MyNetwork.forward(data)))
      self.batchS = temp
    else:
      print(Cal_accuracy(label, MyNetwork.forward(data)))

  def attack_network(self, data, label):
    self.forward(data)
    self.cal_loss(label)
    D_image = np.multiply(expand(self.back_prob_output), self.layer["s" + str(self.number_hidden_layer)])
    for i in range(self.number_hidden_layer):
      D_image = np.matmul(np.transpose(self.weight["w" + str(self.number_hidden_layer - i)]), D_image)
      D_image = np.multiply(self.layer["s" + str(self.number_hidden_layer - i - 1)], D_image)
    D_image = np.matmul(np.transpose(self.weight["w0"]), D_image)
    D_image = np.sign(D_image) 
    return data - self.attack_rate * D_image.reshape((-1,))

  def set_landa(self,x):
    self.landa = x

In [ ]:
MyNetwork = Network(128,1,[150] ,10)

In [ ]:
MyNetwork.set_learning_rate(0.05)
MyNetwork.set_turn(3)
MyNetwork.set_batch_size(128)
MyNetwork.Train_Network(Train_data, Train_label)

In [ ]:
MyNetwork.Test_Network(Train_data, Train_label)

0.815813953488372


In [ ]:
mini_dataSet, mini_labelSet = [],[]
j = 0
for i in range(1000):
  if MyNetwork.forward(Train_data[i]) == Train_label[i]:
    mini_dataSet.append(Train_data[i])
    mini_labelSet.append(Train_label[i])
    j += 1
  if j == 100:
    break

In [ ]:
attacked_image = []
for i in range(100):
  attacked_image.append(MyNetwork.attack_network(mini_dataSet[i], mini_labelSet[i]))

In [ ]:
score1 = 0
score2 = 0
for i in range(100):
  print("The label for the", i,"th data is: ", MyNetwork.forward(mini_dataSet[i]), "after attack it became: ", MyNetwork.forward(attacked_image[i].reshape((-1,))))
  if MyNetwork.forward(mini_dataSet[i]) != MyNetwork.forward(attacked_image[i].reshape((-1,))):
    score1 += 1
  if 10 ==  MyNetwork.forward(attacked_image[i].reshape((-1,))):
    score2 += 1
print("The attacked accuracy was: ", score1)
print("The number rejection is: ", score2)